In [2]:
import numpy as np
import pandas as pd

In [3]:
transactionDataframe = pd.read_csv('../data/transaction_cleaned.csv')

In [20]:
transactionDataframe.assign('TransactionYear'=lambda: df: df.'TransactionData')

,Unnamed: 0,Transaction Number,Transaction Date,Transaction Type,Transaction Description,Debit Amount,Credit Amount,Balance,Category,Location City,Location Country
6064,6064,6065,01/02/2016,FEE,ACCOUNT FEE,9.95,NaN,10748.00,Bills,Swansea,Uk
6066,6066,6067,01/02/2016,DEB,Amazon UK Marketpl,26.55,NaN,10753.38,Amazon,Swansea,Uk
6067,6067,6068,01/02/2016,DEB,Amazon UK Marketpl,19.28,NaN,10779.93,Amazon,Swansea,Uk
6068,6068,6069,01/02/2016,DEB,Amazon UK Marketpl,9.84,NaN,10799.21,Amazon,Swansea,Uk
6069,6069,6070,01/02/2016,DD,NATWEST BANK,828.49,NaN,10809.05,Mortgage,Swansea,Uk
...,...,...,...,...,...,...,...,...,...,...,...
1965,1965,1966,31/12/2020,BP,SAVE THE CHANGE,0.67,NaN,2630.53,Savings,Nottingham,Uk
1964,1964,1965,31/12/2020,DD,TALKTALK LIMITED,18.64,NaN,2611.89,Bills,Nottingham,Uk
784,784,785,31/12/2021,DEB,AMZNMktplace,13.45,NaN,3319.24,Amazon,Nottingham,Uk
785,785,786,31/12/2021,DD,TALKTALK LIMITED,21.00,NaN,3332.69,Bills,Nottingham,Uk


In [ ]:
# helper functions
def parseTransactionDate(transactionDate: str):
    '''
        date format: dd/mm/yyyy,
        return a dictionary: with three keys: 'day', 'month', and 'year', whose values are number
    '''
    [day, month, year]: str[] = transactionDate.split('/')
    return {

    }
